In [1]:
import numpy as np
import pandas as pd
from IPython.display import IFrame

In [2]:
#import streamlit as st
from PIL import Image
# import json
# from Classifier import KNearestNeighbours
from bs4 import BeautifulSoup
import requests,io
import PIL.Image
from urllib.request import urlopen

In [339]:
links = pd.read_csv('resources/data/links.csv')

In [320]:
link.drop('url', axis = 1, inplace=True)

In [340]:
links.head()

,movieId,imdbId,tmdbId,url
0,1,114709,862.0,https://www.imdb.com/title/tt0114709/
1,2,113497,8844.0,https://www.imdb.com/title/tt0113497/
2,3,113228,15602.0,https://www.imdb.com/title/tt0113228/
3,4,114885,31357.0,https://www.imdb.com/title/tt0114885/
4,5,113041,11862.0,https://www.imdb.com/title/tt0113041/


In [347]:
def movie_poster_fetcher(df):
    ## Display Movie Poster
    url_data = requests.get(df).text
    s_data = BeautifulSoup(url_data, 'html.parser')
    imdb_dp = s_data.find("meta", property="og:image")
    movie_poster_link = imdb_dp.attrs['content']
    u = urlopen(movie_poster_link)
    
    return u

In [ ]:
link_list = []
for link in links['url']:
    try:
        link_list.append(movie_poster_fetcher(link))
    except Exception:
        pass
df_link = pd.DataFrame(link_list, columns=['link_url'])
df_link

In [6]:
link['url'] = link['imdbId'].apply(lambda x: "https://www.imdb.com/title/tt" + '0'*(7-len(str(x)))+ str(x) + '/'if len(str(x)) < 7
                                   else "https://www.imdb.com/title/tt" + str(x) + '/')

In [7]:
movie = pd.read_csv('resources/data/movies.csv')

In [8]:
rating = pd.read_csv('resources/data/ratings.csv')

In [176]:
imdb = pd.read_csv('resources/data/imdb_data.csv')

In [9]:
rating.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [178]:
imdb_data = imdb[['movieId', 'director']]#.head()

In [180]:
imdb_data.head()

,movieId,director
0,1,John Lasseter
1,2,Jonathan Hensleigh
2,3,Mark Steven Johnson
3,4,Terry McMillan
4,5,Albert Hackett


In [173]:
rating.shape

(100004, 4)

In [174]:
movie.shape

(62423, 3)

In [10]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [189]:
len(movie['movieId'].unique())

62423

In [11]:
link.head()

,movieId,imdbId,tmdbId,url
0,1,114709,862.0,https://www.imdb.com/title/tt0114709/
1,2,113497,8844.0,https://www.imdb.com/title/tt0113497/
2,3,113228,15602.0,https://www.imdb.com/title/tt0113228/
3,4,114885,31357.0,https://www.imdb.com/title/tt0114885/
4,5,113041,11862.0,https://www.imdb.com/title/tt0113041/


In [158]:
merged = movie.merge(link, how = 'left', on = 'movieId')

In [199]:
merged_director = merged.merge(imdb_data, how = 'left', on = 'movieId')

In [205]:
merged_director['year'] = merged_director['title'].apply(lambda x: x.split(' ')[-1]).str.extract(r'(\d+)')

In [213]:
import random

In [209]:
merged_director.head()

,movieId,title,genres,imdbId,tmdbId,url,director,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,John Lasseter,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,113497,8844.0,https://www.imdb.com/title/tt0113497/,Jonathan Hensleigh,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,113228,15602.0,https://www.imdb.com/title/tt0113228/,Mark Steven Johnson,1995
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,114885,31357.0,https://www.imdb.com/title/tt0114885/,Terry McMillan,1995
4,5,Father of the Bride Part II (1995),Comedy,113041,11862.0,https://www.imdb.com/title/tt0113041/,Albert Hackett,1995


In [215]:
merged_director['genre_list'] = merged_director['genres'].str.split('|')

In [219]:
merged_director['genre'] = merged_director['genre_list'].apply(lambda x: random.choice(x))

In [227]:
merged_director['genre'].unique()

array(['Children', 'Adventure', 'Romance', 'Comedy', 'Crime', 'Action',
       'Horror', 'Animation', 'Drama', 'Mystery', 'Sci-Fi', 'Thriller',
       'IMAX', 'Fantasy', 'Documentary', 'War', 'Western', 'Film-Noir',
       'Musical', '(no genres listed)'], dtype=object)

In [228]:
# merged_data[merged_data['genre'] == '(no genres listed)']

gens = ['Children', 'Adventure', 'Romance', 'Comedy', 'Crime', 'Action',
       'Horror', 'Animation', 'Drama', 'Mystery', 'Sci-Fi', 'Thriller',
       'IMAX', 'Fantasy', 'Documentary', 'War', 'Western', 'Film-Noir',
       'Musical']
   
np.random.seed(1) 
m = merged_director['genre'].eq('(no genres listed)')
merged_director.loc[m, 'genre'] = np.random.choice(gens, size=m.sum())

In [240]:
merged_data = merged_director[['movieId', 'title', 'url', 'director', 'year', 'genre']]

In [242]:
merged_data

,movieId,title,url,director,year,genre
0,1,Toy Story (1995),https://www.imdb.com/title/tt0114709/,John Lasseter,1995,Children
1,2,Jumanji (1995),https://www.imdb.com/title/tt0113497/,Jonathan Hensleigh,1995,Adventure
2,3,Grumpier Old Men (1995),https://www.imdb.com/title/tt0113228/,Mark Steven Johnson,1995,Romance
3,4,Waiting to Exhale (1995),https://www.imdb.com/title/tt0114885/,Terry McMillan,1995,Comedy
4,5,Father of the Bride Part II (1995),https://www.imdb.com/title/tt0113041/,Albert Hackett,1995,Comedy
...,...,...,...,...,...,...
62418,209157,We (2018),https://www.imdb.com/title/tt6671244/,NaN,2018,Drama
62419,209159,Window of the Soul (2001),https://www.imdb.com/title/tt0297986/,NaN,2001,Documentary
62420,209163,Bad Poems (2018),https://www.imdb.com/title/tt6755366/,NaN,2018,Drama
62421,209169,A Girl Thing (2001),https://www.imdb.com/title/tt0249603/,NaN,2001,Film-Noir


In [284]:
merged_data[merged_data['year'] == '1995']

,movieId,title,url,director,year,genre
0,1,Toy Story (1995),https://www.imdb.com/title/tt0114709/,John Lasseter,1995,Children
1,2,Jumanji (1995),https://www.imdb.com/title/tt0113497/,Jonathan Hensleigh,1995,Adventure
2,3,Grumpier Old Men (1995),https://www.imdb.com/title/tt0113228/,Mark Steven Johnson,1995,Romance
3,4,Waiting to Exhale (1995),https://www.imdb.com/title/tt0114885/,Terry McMillan,1995,Comedy
4,5,Father of the Bride Part II (1995),https://www.imdb.com/title/tt0113041/,Albert Hackett,1995,Comedy
...,...,...,...,...,...,...
61444,205933,August in the Water (1995),https://www.imdb.com/title/tt0113834/,NaN,1995,Sci-Fi
61597,206327,Scanners: The Showdown (1995),https://www.imdb.com/title/tt0114344/,NaN,1995,Horror
61701,206682,Dalekmania (1995),https://www.imdb.com/title/tt0416673/,NaN,1995,Documentary
62349,208821,"Fireworks, Should We See It from the Side or t...",https://www.imdb.com/title/tt0217862/,NaN,1995,Drama


In [317]:
col = np.where((merged_data['year'] == '2016') | (merged_data['genre'] == 'Documentary')
               | (merged_data['director'] == 'Agnès de Sacy'))

In [307]:
sorted_rating = rating.sort_values('rating', ascending=False).head(100)

In [308]:
sorted_rating

,userId,movieId,rating,timestamp
33889,242,2929,5.0,956687566
36867,265,1233,5.0,960056214
46251,337,1356,5.0,1447176421
14749,95,3948,5.0,1025556197
14747,95,3917,5.0,1019023102
...,...,...,...,...
14878,96,32525,5.0,1223256595
77799,538,1,5.0,831835670
14876,96,25947,5.0,1223256567
14874,96,25850,5.0,1223256486


In [318]:
merged_data.iloc[col]  

,movieId,title,url,director,year,genre
76,77,Nico Icon (1995),https://www.imdb.com/title/tt0113973/,Susanne Ofteringer,1995,Documentary
97,99,Heidi Fleiss: Hollywood Madam (1995),https://www.imdb.com/title/tt0113283/,Nick Broomfield,1995,Documentary
106,108,Catwalk (1996),https://www.imdb.com/title/tt0112646/,Robert Leacock,1996,Documentary
114,116,Anne Frank Remembered (1995),https://www.imdb.com/title/tt0112373/,Jon Blair,1995,Documentary
126,128,Jupiter's Wife (1994),https://www.imdb.com/title/tt0110217/,Michel Negroponte,1994,Documentary
...,...,...,...,...,...,...
62404,209123,Square Roots: The Story of SpongeBob SquarePan...,https://www.imdb.com/title/tt1489931/,NaN,2009,Documentary
62405,209129,Destination Titan (2011),https://www.imdb.com/title/tt2361219/,NaN,2011,Documentary
62406,209131,Last Days of the Arctic (2011),https://www.imdb.com/title/tt2040407/,NaN,2011,Documentary
62408,209135,Jane B. by Agnès V. (1988),https://www.imdb.com/title/tt0093295/,NaN,1988,Documentary


In [311]:
url_links = pd.DataFrame()
movieIds = [movieId for movieId in merged_data['movieId'] if movieId in sorted_rating['movieId']]
#url_links['url'] = [merged_data['url'] for merged_data['mo']]

In [315]:
movieIds
suggested = merged_data[merged_data['movieId'].isin(movieIds)]['url']

In [316]:
suggested

14642    https://www.imdb.com/title/tt0338977/
14645    https://www.imdb.com/title/tt0074857/
14646    https://www.imdb.com/title/tt1329454/
14648    https://www.imdb.com/title/tt0892074/
14649    https://www.imdb.com/title/tt0046424/
14650    https://www.imdb.com/title/tt0054380/
14660    https://www.imdb.com/title/tt1179056/
16796    https://www.imdb.com/title/tt0783695/
16805    https://www.imdb.com/title/tt0031282/
Name: url, dtype: object

In [310]:
url_links

,url
0,1
1,2
2,3
3,4


In [241]:
merged_data.to_csv('merged_data.csv', index=False, encoding='utf8')

In [210]:
merged_director_genres = merged_director['genres'].str.split('|').apply(pd.Series, 1).stack()

In [211]:
merged_director_genres

0      0             Adventure
       1             Animation
       2              Children
       3                Comedy
       4               Fantasy
                   ...        
62420  1                 Drama
62421  0    (no genres listed)
62422  0                Action
       1             Adventure
       2                 Drama
Length: 112307, dtype: object

In [200]:
len(merged_director['director'].unique())#.tail()

10494

In [201]:
len(merged_director['movieId'].unique())

62423

In [161]:
merged_ratings = merged.merge(rating, how = 'left', on = 'movieId')

In [243]:
merged_ratings.head()

,movieId,title,genres,imdbId,tmdbId,url,userId,rating,timestamp,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,7.0,3.0,8.518667e+08,1995
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,9.0,4.0,9.386292e+08,1995
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,13.0,5.0,1.331380e+09,1995
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,15.0,2.0,9.979383e+08,1995
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,114709,862.0,https://www.imdb.com/title/tt0114709/,19.0,3.0,8.551901e+08,1995


In [244]:
merged_ratings['year'] = merged_ratings['title'].apply(lambda x: x.split(' ')[-1]).str.extract(r'(\d+)')

In [255]:
most = merged_ratings.sort_values(['rating', 'year'], axis=0, ascending=[False, False])

In [260]:
most_url = most[['url', 'rating', 'year']]

In [265]:
most_url['year'].fillna(0).astype(int)

115452    2016
115455    2016
115498    2016
115500    2016
115502    2016
          ... 
152770       0
152803       0
152984       0
153025       0
153079       0
Name: year, Length: 153122, dtype: int32

In [266]:
most_url

,url,rating,year
115452,https://www.imdb.com/title/tt2638144/,5.0,2016
115455,https://www.imdb.com/title/tt0803096/,5.0,2016
115498,https://www.imdb.com/title/tt1431045/,5.0,2016
115500,https://www.imdb.com/title/tt1431045/,5.0,2016
115502,https://www.imdb.com/title/tt1431045/,5.0,2016
...,...,...,...
152770,https://www.imdb.com/title/tt11101550/,NaN,NaN
152803,https://www.imdb.com/title/tt4602718/,NaN,NaN
152984,https://www.imdb.com/title/tt3736766/,NaN,NaN
153025,https://www.imdb.com/title/tt0098693/,NaN,NaN


In [267]:
most_url.to_csv('most_recent.csv', index=False, encoding='utf8')

In [170]:
m = merged_ratings.fillna(0)#.sort_values(by='year', axis =0, ascending=False)#unique()#.head()

In [172]:
m[m['rating'] == 0]

,movieId,title,genres,imdbId,tmdbId,url,userId,rating,timestamp,year
1849,33,Wings of Courage (1995),Adventure|Romance|IMAX,114952,78802.0,https://www.imdb.com/title/tt0114952/,0.0,0.0,0.0,1995
2845,51,Guardian Angel (1994),Action|Drama|Thriller,109950,117164.0,https://www.imdb.com/title/tt0109950/,0.0,0.0,0.0,1994
2907,56,Kids of the Round Table (1995),Adventure|Children|Comedy|Fantasy,113541,124057.0,https://www.imdb.com/title/tt0113541/,0.0,0.0,0.0,1995
3136,67,Two Bits (1995),Drama,114753,40628.0,https://www.imdb.com/title/tt0114753/,0.0,0.0,0.0,1995
3260,75,Big Bully (1996),Comedy|Drama,115676,36929.0,https://www.imdb.com/title/tt0115676/,0.0,0.0,0.0,1996
...,...,...,...,...,...,...,...,...,...,...
153117,209157,We (2018),Drama,6671244,499546.0,https://www.imdb.com/title/tt6671244/,0.0,0.0,0.0,2018
153118,209159,Window of the Soul (2001),Documentary,297986,63407.0,https://www.imdb.com/title/tt0297986/,0.0,0.0,0.0,2001
153119,209163,Bad Poems (2018),Comedy|Drama,6755366,553036.0,https://www.imdb.com/title/tt6755366/,0.0,0.0,0.0,2018
153120,209169,A Girl Thing (2001),(no genres listed),249603,162892.0,https://www.imdb.com/title/tt0249603/,0.0,0.0,0.0,2001


In [149]:
# merged['year'].unique()
# Fill NaN values with 0 and convert the years to integers.
merged['year'] = merged['year'].fillna(0).astype(int)

# Fish out the odd years
odd_ones = [0, 2, 3, 5, 6, 25, 101, 261]

# Slice the merged data frame by removing the records with the odd year.
merged = merged[~merged['year'].isin(odd_ones)]

In [150]:
merged['year'].unique()

array([1995, 1994, 1996, 1976, 1992, 1988, 1967, 1993, 1964, 1977, 1965,
       1982, 1990, 1991, 1989, 1937, 1940, 1969, 1981, 1973, 1970, 1960,
       1955, 1959, 1968, 1980, 1975, 1986, 1948, 1943, 1950, 1946, 1987,
       1997, 1974, 1956, 1958, 1949, 1972, 1998, 1933, 1952, 1951, 1957,
       1961, 1954, 1934, 1944, 1963, 1942, 1941, 1953, 1939, 1947, 1945,
       1938, 1935, 1936, 1926, 1932, 1985, 1979, 1971, 1978, 1966, 1962,
       1983, 1984, 1931, 1922, 1999, 1927, 1929, 1930, 1928, 1925, 1914,
       2000, 1919, 1923, 1920, 1918, 1921, 2001, 1924, 2002, 2003, 1915,
       2004, 1916, 1917, 2005, 2006, 1902, 1903, 2007, 2008, 2009, 1912,
       2010, 1913, 2011, 1898, 1899, 1894, 2012, 1910, 2013, 1896, 2014,
       2015, 1895, 1909, 1911, 1900, 2016, 2017, 2018, 2019, 1905, 1904,
       1891, 1892, 1908, 1897, 1887, 1888, 1890, 1878, 1874, 1901, 1907,
       1906, 1883, 1880])

In [117]:
merged.to_csv('merged_data.csv', index=False, encoding='utf8')

In [14]:
# merged['year'] = merged['title'].str.extract(r'(\d+)')

In [118]:
merged['year'] = merged['year'].fillna(0).astype(int)

In [122]:
merged['year']

0        1995
1        1995
2        1995
3        1995
4        1995
         ... 
62418    2018
62419    2001
62420    2018
62421    2001
62422    1962
Name: year, Length: 62423, dtype: int32

In [124]:
merged[merged['year']== 0]# or merged['year']== 101 or merged['year']== 6 or merged['year']== 2]

,movieId,title,genres,imdbId,tmdbId,url,year
8463,25936,"Babe Ruth Story, The (1948)",Drama,40142,43452.0,https://www.imdb.com/title/tt0040142/,0
8602,26137,"Heroes of Telemark, The (1965)",Action|Drama|War,59263,16850.0,https://www.imdb.com/title/tt0059263/,0
8910,26628,Jack's Back (1988),Crime|Horror|Mystery|Thriller,95389,41959.0,https://www.imdb.com/title/tt0095389/,0
9120,27189,After the Rain (Ame agaru) (1999),Action|Drama,181960,15319.0,https://www.imdb.com/title/tt0181960/,0
9763,32497,Love Letter (1995),Drama|Romance,113703,47002.0,https://www.imdb.com/title/tt0113703/,0
...,...,...,...,...,...,...,...
62071,207714,Tales of Found Footage,(no genres listed),11101550,640427.0,https://www.imdb.com/title/tt11101550/,0
62104,207884,Enduring Destiny,(no genres listed),4602718,528815.0,https://www.imdb.com/title/tt4602718/,0
62285,208597,Punk the Capital: Building a Sound Movement,Documentary,3736766,636391.0,https://www.imdb.com/title/tt3736766/,0
62326,208763,Yosemite: The Fate of Heaven,(no genres listed),98693,352556.0,https://www.imdb.com/title/tt0098693/,0


In [35]:
year_data = pd.DataFrame(merged['year'].unique(), columns=['year'])

In [66]:
year_data.sort_values(by='year', axis =0, ascending=False, inplace=True)

In [67]:
year_data

,year
119,2019
118,2018
117,2017
116,2016
111,2015
...,...
127,1887
137,1883
141,1880
130,1878


In [68]:
year_data.to_csv('year_data.csv', index = False, encoding = 'utf8')#.drop(2, inplace=True)

In [81]:
sorted_merged = merged.sort_values('year', axis=0, ascending=False)

In [70]:
movie.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [74]:
genre = pd.DataFrame(movie['genres'].apply(lambda x: x.split("|")))

In [90]:
genres = movie['genres'].str.split('|').apply(pd.Series, 1).stack()

In [91]:
genres = pd.DataFrame(genres, columns=['genre'])

In [95]:
genres = genres.reset_index()

In [107]:
genre_data = pd.DataFrame(genres['genre'].unique(), columns=['genre']).sort_values('genre', axis = 0, ascending=True)

In [109]:
genre_data.drop(19, inplace=True)

In [111]:
genre_data.to_csv('genre_data.csv', index = False, encoding = 'utf8')

In [110]:
sorted_merged.to_csv('most_recent.csv', index = False, encoding = 'utf8')

In [93]:
sampled_recent = most_recent.head(20).sample(3)

In [95]:
for link in sampled_recent['url']:
    print(link)

https://www.imdb.com/title/tt9758270/
https://www.imdb.com/title/tt4638148/
https://www.imdb.com/title/tt9426212/


In [134]:
sampled_recent['year']

60953    2019
58424    2019
58289    2019
Name: year, dtype: int32

In [131]:
# df_merged = merged.drop(merged.index[[num for num in merged['year'] if len(str(num)) < 4]], axis = 0) #unique() #.groupby('year').count()

In [129]:
odd = [num for num in merged['year'] if len(str(num)) < 4]

year = list(set([record for record in merged['year'] if record not in odd]))

In [132]:
# merged[merged['year'] == year]

In [97]:
merged[merged['year'] == '101']

,movieId,title,genres,imdbId,tmdbId,url,year
36122,149502,101次求婚,(no genres listed),1674766,187006.0,https://www.imdb.com/title/tt01674766/,101


In [9]:
# data['url'] = data['imdbId'].apply(lambda x: "https://www.imdb.com/title/tt0" + str(x) + '/')

In [10]:
data.shape

(62423, 4)

In [7]:
#pip install streamlit

In [ ]:
https://www.imdb.com/video/vi2052129305?playlistId=tt0114709&ref_=tt_ov_vi
    
https://www.imdb.com/video/vi2171864857/?playlistId=tt0114709?ref_=ext_shr_lnk

In [21]:
# IFrame(width="840", height="472", src="https://youtu.be/8MNJl0L2xe8")

In [4]:
#IFrame(width="840", height="472", src="https://www.youtube.com/embed/NEaUSP4YerM")

In [97]:
def movie_poster_fetcher(df):
    ## Display Movie Poster
    url_data = requests.get(df).text
    s_data = BeautifulSoup(url_data, 'html.parser')
    imdb_dp = s_data.find("meta", property="og:image")
    movie_poster_link = imdb_dp.attrs['content']
    u = urlopen(movie_poster_link)
    raw_data = u.read()
    image = PIL.Image.open(io.BytesIO(raw_data))
    image = image.resize((158, 301), )
    image.show()
#   st.image(image, use_column_width=False)

In [98]:
for link in sampled_recent['url']:
    movie_poster_fetcher(link)

In [130]:
def get_movie_info(df):
    url_data = requests.get(df).text
    s_data = BeautifulSoup(url_data, 'html.parser')
    imdb_content = s_data.find("meta", property="og:description")
    movie_descr = imdb_content.attrs['content']
    movie_descr = str(movie_descr).split('.')
    movie_director = movie_descr[0]
    movie_cast = str(movie_descr[1])#.replace('With', 'Cast: ').strip()
    movie_story = str(movie_descr[2]).strip()+'.'
#   movie_story = 'Story: ' + str(movie_descr[2]).strip()+'.'
#   rating = s_data.find("div", class_="AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ")
#   rating = str(rating).split('<div class="AggregateRatingButton__TotalRatingAmount-sc-1ll29m0-3 jkCVKJ')
#   rating = str(rating[1]).split("</div>")
#   rating = str(rating[0]).replace(''' "> ''', '').replace('">', '')

    #movie_rating = 'Total Rating count: '+ rating
    director = movie_director.split(":")[-1]
    title = ':'.join(movie_director.split(":")[0:-1])
    cast = movie_cast
    story = movie_story
#   movie_director,movie_cast,movie_story
    return {'Director': director, 
           'Title': title,
           'Cast': cast,
           'Story': story}

In [132]:
info = []
for link in sampled_recent['url']:
    info.append(get_movie_info(link))
info[2]

{'Director': ' Directed by Manny Rodriguez',
 'Title': 'Gabriel "Fluffy" Iglesias: One Show Fits All',
 'Cast': ' With Gabriel Iglesias',
 'Story': 'Gabriel "Fluffy" Iglesias discusses his teenage son, encounters with Snoop Dogg and an overzealous fan, and more.'}

In [126]:
word = 'Braid: Directed by Mitzi Peirone'.split(":")[0:-1]

In [127]:
':'.join(word)

'Braid'

In [6]:
get_movie_info(link)

('The Shawshank Redemption: Directed by Frank Darabont',
 'Cast:  Tim Robbins, Morgan Freeman, Bob Gunton, William Sadler',
 'Story: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.')

In [36]:
# page = requests.get("https://www.imdb.com/title/tt0114709/")
# bSoup = BeautifulSoup(page.content, 'html.parser')
# lis_nklist = bSoup.find_all('a')

# for link in links_list:
#     if 'href' in link.attrs and '.mp4' in link.split('@'):
#         print(str(link.attrs['href']) + "\n")